In [1]:
import pandas as pd
import numpy as np
import csv

df = pd.read_csv('nasdaq/AppleInc.csv', index_col = 'Date', parse_dates=True)
iter_len = len(df['Close'])


# Labeling: increase = 1, decrease = -1

with open('Apple.csv', 'w', newline='') as f:
    wt = csv.writer(f)
    wt.writerow(['avg'])
    #wt.writerow("Date,Open,High,Low,Close,Adj Close,Volume,Label".split(","))
    for i in range(iter_len):
        line1 = df.iloc[i]
        a1 = (line1['Open']+line1['High']+line1['Low']+line1['Close'])/4
        wt.writerow([a1])

In [2]:
df = pd.read_csv('Apple.csv')['avg']
iter_len = len(df)


with open('Apple-1.csv', 'w', newline='') as f:
    wt = csv.writer(f)
    wt.writerow("1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,label".split(","))
    for i in range(iter_len-21):
        line1 = [df.iloc[j] for j in range(i,i+20)]
        M=max(line1)
        m=min(line1)
        new_line = [(x-m)/(M-m) for x in line1]
        if ( df.iloc[i+19] < df.iloc[i+20] ):
            new_line.append(1)
        else: new_line.append(0)
        wt.writerow(new_line)


In [3]:
df = pd.read_csv('Apple-1.csv')
print(df)

              1         2         3         4         5         6         7  \
0      0.323616  0.186598  0.000000  0.055390  0.125378  0.276971  0.405249   
1      0.186598  0.000000  0.055390  0.125378  0.276971  0.405249  0.521875   
2      0.000000  0.055390  0.125378  0.276971  0.405249  0.521875  0.673469   
3      0.000000  0.074091  0.234574  0.370374  0.493839  0.654322  0.950621   
4      0.000000  0.173325  0.319991  0.453335  0.626661  0.946669  1.000000   
...         ...       ...       ...       ...       ...       ...       ...   
10282  0.720465  0.703338  0.558491  0.296662  0.301887  0.274021  0.150943   
10283  0.703338  0.558491  0.296662  0.301887  0.274021  0.150943  0.000000   
10284  0.537130  0.285316  0.290340  0.263540  0.145170  0.000000  0.022892   
10285  0.285316  0.290340  0.263540  0.145170  0.000000  0.022892  0.017029   
10286  0.280172  0.254310  0.140086  0.000000  0.022091  0.016433  0.320851   

              8         9        10  ...        12 

In [4]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [5]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

newY=[]
for val in y:
    if val == 1:
        newY.append(np.array([1,0]))
    else: newY.append(np.array([0,1]))

In [6]:
x=np.array(X)
y=np.array(newY)

if torch.cuda.is_available():
    x=torch.tensor(x).cuda()
    y=torch.tensor(y).cuda()
    print("CUDA available")
else:
    x=torch.tensor(x)
    y=torch.tensor(y)

In [15]:
class Net(torch.nn.Module):
    def __init__(self, D_in, D_out):
        super(Net, self).__init__()
        self.l1 = nn.Linear(D_in, D_in)
        self.h1 = nn.Linear(D_in, round(D_in/2))
        self.h2 = nn.Linear(round(D_in/2),round(D_in/4))
        self.l2 = nn.Linear(round(D_in/4), D_out)

    def forward(self, x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.h1(x))
        x = F.leaky_relu(self.h2(x))
        y = F.softmax(self.l2(x))
        return y


D_in, D_out = 20, 2 #input, output 

def run(lr, epochs):
  
    #training
    
    model = Net(D_in, D_out)
    if torch.cuda.is_available(): model = model.cuda()
    criterion = torch.nn.MSELoss(reduction='sum')
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    for t in range(epochs):
        y_pred = model(x.float())
        loss = criterion(y_pred, y.float())
        print(t, loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

  ###############################################################################
  
    #evaluation
    total=0
    correct=0
    
    for idx, inp in enumerate(x):
        total+=1
        y_pred = list(model(inp.float()).cpu().detach().numpy())
        y_truth = list(newY[idx])
        print(y_pred, y_truth)
        if y_pred.index(max(y_pred))==y_truth.index(max(y_truth)): correct+=1
    print(round(100*correct/total,2))


In [16]:
run(1e-3, 1000)

<ipython-input-15-5781b7e9ce4a>:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y = F.softmax(self.l2(x))


0 5312.11279296875
1 7698.45703125
2 7358.9013671875
3 8753.5146484375
4 5646.1357421875
5 8591.775390625
6 6398.28759765625
7 9119.0107421875
8 5152.59814453125
9 5410.46923828125
10 7663.70703125
11 7193.63427734375
12 8864.96875
13 6100.529296875
14 9102.9609375
15 5163.81640625
16 5581.93310546875
17 8268.9716796875
18 5350.8525390625
19 7344.40625
20 8038.57568359375
21 7853.458984375
22 6584.62109375
23 9045.3720703125
24 7012.11865234375
25 8635.328125
26 6263.6220703125
27 9133.4296875
28 5144.89599609375
29 5287.0107421875
30 6902.7119140625
31 8774.919921875
32 5841.21630859375
33 8913.8984375
34 5472.306640625
35 8060.6044921875
36 7808.6708984375
37 6732.7783203125
38 9031.3583984375
39 6937.4033203125
40 8733.4248046875
41 5963.7626953125
42 9028.8759765625
43 5249.103515625
44 6620.0654296875
45 9043.7314453125
46 7003.36572265625
47 8647.4375
48 6226.1943359375
49 9131.3232421875
50 5145.87353515625
51 5302.9794921875
52 7024.8701171875
53 8617.33203125
54 6319.249511718

432 7556.6044921875
433 7506.1025390625
434 8624.916015625
435 5289.7265625
436 6988.5166015625
437 8962.9052734375
438 6578.5205078125
439 9048.6962890625
440 5221.11083984375
441 6324.373046875
442 9021.30859375
443 6884.083984375
444 8795.9658203125
445 5780.5068359375
446 8835.0400390625
447 5671.85400390625
448 8646.9736328125
449 6227.42578125
450 9131.294921875
451 5145.8505859375
452 5302.8544921875
453 7023.96240234375
454 8618.42578125
455 6315.66259765625
456 9131.6748046875
457 5145.669921875
458 5299.9150390625
459 7002.10791015625
460 8648.966796875
461 6221.2705078125
462 9130.697265625
463 5146.13671875
464 5307.52099609375
465 7058.083984375
466 8568.576171875
467 6469.53857421875
468 9097.9189453125
469 5167.841796875
470 5641.388671875
471 8408.5263671875
472 5156.66552734375
473 5469.4150390625
474 8048.697265625
475 7832.80224609375
476 6652.89697265625
477 9041.140625
478 6989.64453125
479 8665.8994140625
480 6169.06591796875
481 9122.509765625
482 5150.4711914062

870 5251.16162109375
871 6640.4638671875
872 9042.1435546875
873 6995.03466796875
874 8658.51953125
875 6191.70068359375
876 9126.6865234375
877 5148.1484375
878 5340.13525390625
879 7278.35888671875
880 8179.154296875
881 7547.265625
882 7531.90283203125
883 8599.1884765625
884 5242.97412109375
885 6558.59765625
886 9045.705078125
887 7014.0546875
888 8632.298828125
889 6272.658203125
890 9133.2861328125
891 5144.90185546875
892 5287.5302734375
893 6906.8662109375
894 8769.779296875
895 5855.90234375
896 8930.3740234375
897 5434.5263671875
898 7895.67236328125
899 8116.29296875
900 5735.49609375
901 8582.375
902 5216.99267578125
903 6277.609375
904 9009.427734375
905 6821.22900390625
906 8862.357421875
907 5599.26708984375
908 8477.728515625
909 6745.06494140625
910 8932.5625
911 5429.65625
912 7872.7158203125
913 8154.3232421875
914 5627.341796875
915 8377.859375
916 5182.86376953125
917 5837.759765625
918 8909.642578125
919 5482.05859375
920 8099.4931640625
921 7726.4560546875
922 6

[0.8204208, 0.1795792] [0, 1]
[0.8204276, 0.17957243] [0, 1]
[0.8204158, 0.17958423] [0, 1]
[0.8204133, 0.17958672] [0, 1]
[0.820426, 0.17957404] [0, 1]
[0.82043195, 0.17956811] [0, 1]
[0.8204268, 0.17957324] [1, 0]
[0.8204281, 0.17957193] [1, 0]
[0.8204317, 0.17956828] [1, 0]
[0.8204438, 0.17955619] [1, 0]
[0.82043797, 0.17956205] [1, 0]
[0.82043135, 0.17956866] [1, 0]
[0.8204406, 0.17955936] [1, 0]
[0.82043916, 0.17956088] [1, 0]
[0.8204292, 0.17957085] [1, 0]
[0.8204327, 0.17956723] [0, 1]
[0.82043, 0.17956999] [0, 1]
[0.8204225, 0.17957756] [0, 1]
[0.82041967, 0.17958035] [1, 0]
[0.82042414, 0.17957586] [0, 1]
[0.82042927, 0.17957069] [1, 0]
[0.82043016, 0.1795698] [0, 1]
[0.82043123, 0.17956881] [0, 1]
[0.8204338, 0.1795662] [1, 0]
[0.82043856, 0.17956138] [1, 0]
[0.8204381, 0.17956187] [1, 0]
[0.8204348, 0.1795652] [0, 1]
[0.82043207, 0.17956787] [0, 1]
[0.820428, 0.179572] [0, 1]
[0.8204295, 0.17957045] [0, 1]
[0.8204283, 0.1795717] [1, 0]
[0.8204171, 0.17958295] [0, 1]
[0.82041

...

[0.8204475, 0.1795525] [1, 0]
[0.8204316, 0.17956838] [1, 0]
[0.8204158, 0.17958419] [1, 0]
[0.8204316, 0.17956842] [1, 0]
[0.8204346, 0.17956543] [1, 0]
[0.8204315, 0.17956857] [1, 0]
[0.82043505, 0.17956494] [1, 0]
[0.8204409, 0.17955911] [1, 0]
[0.8204296, 0.17957039] [1, 0]
[0.82042474, 0.17957531] [0, 1]
[0.8204327, 0.1795673] [0, 1]
[0.82042617, 0.1795738] [0, 1]
[0.8204221, 0.1795779] [1, 0]
[0.82043195, 0.17956805] [0, 1]
[0.8204315, 0.17956851] [1, 0]
[0.8204354, 0.1795646] [1, 0]
[0.8204403, 0.17955968] [0, 1]
[0.82043546, 0.17956452] [0, 1]
[0.820437, 0.17956302] [1, 0]
[0.8204361, 0.1795639] [1, 0]
[0.8204392, 0.17956075] [1, 0]
[0.82043546, 0.17956449] [0, 1]
[0.8204339, 0.17956606] [1, 0]
[0.82043964, 0.1795604] [1, 0]
[0.8204236, 0.17957643] [0, 1]
[0.820425, 0.17957506] [0, 1]
[0.82042557, 0.17957439] [0, 1]
[0.8204102, 0.17958981] [1, 0]
[0.8204251, 0.17957488] [1, 0]
[0.82042384, 0.17957619] [1, 0]
[0.8204252, 0.17957482] [1, 0]
[0.8204289, 0.17957109] [1, 0]
[0.82043

[0.8204359, 0.17956413] [1, 0]
[0.8204419, 0.17955808] [0, 1]
[0.8204311, 0.17956892] [0, 1]
[0.8204216, 0.17957838] [0, 1]
[0.8204219, 0.17957807] [0, 1]
[0.8204144, 0.17958556] [1, 0]
[0.8204385, 0.17956154] [1, 0]
[0.8204359, 0.17956418] [1, 0]
[0.82041186, 0.17958808] [1, 0]
[0.8204274, 0.17957251] [0, 1]
[0.8204264, 0.17957363] [1, 0]
[0.8204285, 0.17957152] [1, 0]
[0.8204345, 0.17956555] [0, 1]
[0.8204292, 0.17957085] [0, 1]
[0.8204215, 0.17957845] [1, 0]
[0.82040924, 0.17959076] [0, 1]
[0.82041854, 0.17958148] [0, 1]
[0.82041466, 0.17958531] [1, 0]
[0.8204134, 0.17958657] [1, 0]
[0.8204272, 0.17957276] [1, 0]
[0.820441, 0.17955896] [1, 0]
[0.82044125, 0.1795588] [0, 1]
[0.82042986, 0.17957018] [0, 1]
[0.8204274, 0.17957258] [1, 0]
[0.8204325, 0.17956749] [1, 0]
[0.8204248, 0.17957523] [1, 0]
[0.82043636, 0.17956364] [1, 0]
[0.82043576, 0.17956424] [1, 0]
[0.82042575, 0.17957422] [1, 0]
[0.8204278, 0.17957221] [0, 1]
[0.82042944, 0.17957056] [0, 1]
[0.8204263, 0.17957368] [0, 1]


[0.8204382, 0.17956181] [1, 0]
[0.8204223, 0.17957775] [1, 0]
[0.8204186, 0.17958142] [1, 0]
[0.8204375, 0.17956254] [0, 1]
[0.8204374, 0.17956257] [0, 1]
[0.8204198, 0.17958021] [0, 1]
[0.8204132, 0.17958684] [0, 1]
[0.82040924, 0.17959079] [1, 0]
[0.82042974, 0.1795702] [1, 0]
[0.8204334, 0.17956662] [0, 1]
[0.820435, 0.17956501] [0, 1]
[0.82043, 0.17956996] [1, 0]
[0.82042223, 0.17957775] [0, 1]
[0.8204315, 0.17956851] [0, 1]
[0.82042646, 0.17957349] [0, 1]
[0.8204177, 0.17958227] [0, 1]
[0.82041734, 0.17958273] [0, 1]
[0.82042056, 0.17957945] [0, 1]
[0.82042617, 0.1795739] [1, 0]
[0.8204276, 0.17957245] [1, 0]
[0.8204273, 0.1795727] [0, 1]
[0.820432, 0.17956801] [0, 1]
[0.8204266, 0.17957349] [1, 0]
[0.82042944, 0.17957053] [1, 0]
[0.82043356, 0.17956652] [0, 1]
[0.82042474, 0.17957531] [1, 0]
[0.8204236, 0.1795764] [1, 0]
[0.82042223, 0.17957775] [1, 0]
[0.8204221, 0.17957787] [1, 0]
[0.8204283, 0.17957167] [1, 0]
[0.8204183, 0.17958167] [1, 0]
[0.8204239, 0.17957607] [1, 0]
[0.82

[0.820432, 0.17956801] [0, 1]
[0.8204274, 0.1795726] [0, 1]
[0.8204199, 0.17958008] [1, 0]
[0.82043564, 0.17956439] [0, 1]
[0.8204304, 0.17956957] [0, 1]
[0.82042134, 0.17957865] [1, 0]
[0.82041925, 0.17958075] [1, 0]
[0.8204268, 0.1795732] [1, 0]
[0.8204373, 0.1795627] [1, 0]
[0.820426, 0.17957404] [0, 1]
[0.8204165, 0.17958352] [0, 1]
[0.82041585, 0.17958416] [0, 1]
[0.8204231, 0.17957689] [0, 1]
[0.8204308, 0.17956914] [0, 1]
[0.82042944, 0.17957053] [1, 0]
[0.82043785, 0.17956214] [1, 0]
[0.8204337, 0.17956626] [0, 1]
[0.8204309, 0.17956913] [1, 0]
[0.8204377, 0.17956233] [1, 0]
[0.8204302, 0.17956975] [1, 0]
[0.82043433, 0.17956564] [1, 0]
[0.8204443, 0.1795557] [1, 0]
[0.8204457, 0.17955425] [1, 0]
[0.8204242, 0.17957579] [1, 0]
[0.8204273, 0.17957269] [0, 1]
[0.82042366, 0.17957635] [0, 1]
[0.82041013, 0.17958993] [1, 0]
[0.8204208, 0.1795792] [1, 0]
[0.8204344, 0.17956553] [1, 0]
[0.8204309, 0.17956908] [0, 1]
[0.8204229, 0.17957719] [0, 1]
[0.8204263, 0.17957366] [1, 0]
[0.820

[0.8204318, 0.17956817] [1, 0]
[0.82042783, 0.17957217] [0, 1]
[0.82042706, 0.17957294] [0, 1]
[0.82042325, 0.1795767] [1, 0]
[0.8204246, 0.17957541] [1, 0]
[0.8204324, 0.17956756] [1, 0]
[0.82043344, 0.17956653] [1, 0]
[0.8204392, 0.1795608] [1, 0]
[0.82043684, 0.17956321] [0, 1]
[0.8204414, 0.17955865] [0, 1]
[0.82042944, 0.17957056] [1, 0]
[0.82042783, 0.17957218] [0, 1]
[0.8204299, 0.17957011] [1, 0]
[0.82043594, 0.17956406] [1, 0]
[0.8204369, 0.17956308] [1, 0]
[0.8204327, 0.17956722] [0, 1]
[0.8204284, 0.17957157] [0, 1]
[0.820422, 0.179578] [0, 1]
[0.8204281, 0.17957188] [1, 0]
[0.8204425, 0.17955744] [0, 1]
[0.8204352, 0.1795648] [1, 0]
[0.82043827, 0.1795617] [1, 0]
[0.82043517, 0.17956483] [1, 0]
[0.8204259, 0.17957406] [1, 0]
[0.82041967, 0.17958036] [0, 1]
[0.82041085, 0.17958918] [0, 1]
[0.82041883, 0.17958112] [1, 0]
[0.8204237, 0.1795762] [1, 0]
[0.8204308, 0.1795692] [1, 0]
[0.8204348, 0.17956516] [0, 1]
[0.82042325, 0.17957674] [0, 1]
[0.82041574, 0.17958425] [0, 1]
[0